Contextual retrieval in the OpenAI LangChain Python library is a technique that combines semantic search and natural 

language processing to improve the relevance of retrieved information from a knowledge base or corpus of documents.

when we give query it give us answer but most of time that query related respone we dont able to get properly 
we get output but it is not that much impactfull beoz of huge data  

and pass the full document it also expensive 

these problem solve by contextual retrival 

by using that we make model response good and contextual 


In [23]:
from decouple import config 
secrete_key = config('OPENAI_API_KEY') 

In [30]:
from langchain_openai import ChatOpenAI 
chat=ChatOpenAI(openai_api_key=secrete_key) 

In [24]:
from langchain_openai import OpenAIEmbeddings  
models=OpenAIEmbeddings (openai_api_key=secrete_key)

In [25]:
from langchain_openai import ChatOpenAI 
from langchain_community.document_loaders import TextLoader 
from langchain_openai import OpenAIEmbeddings 
from langchain.text_splitter import CharacterTextSplitter 
from langchain_community.vectorstores import Chroma   


from langchain.retrievers import ContextualCompressionRetriever  
from langchain.retrievers.document_compressors import LLMChainExtractor   

In [26]:
# Load data 

Loader=TextLoader('E:\\Langchain_Prac\\9.contextual_retrival\\notes.txt') 
my_data= Loader.load()  

# split documnet 
text_splitter =CharacterTextSplitter(chunk_size=600 , chunk_overlap=0)   

my_document = text_splitter.split_documents(my_data) 

# embeding_model 
model_emb =OpenAIEmbeddings(openai_api_key=secrete_key)  

# store 
db=Chroma.from_documents(my_document , model_emb , persist_directory="./chroma_db1") 

db.persist()  

In [27]:
# read_from chroma db   

db_connections = Chroma(embedding_function= model_emb , persist_directory="./chroma_db1" ) 

query = " in whcih years IPL matches hosted in United Arab Emirates ?"  

# vector store retrival 

retrival = db_connections.as_retriever(search_kwargs={"k":1}) 

similar_doc =retrival.get_relevant_documents(query)  

print(similar_doc[0].page_content) 


#  LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 
# and will be removed in 0.3.0. Use invoke instead.

2. **Shift to UAE**:
   Due to the COVID-19 pandemic, the IPL has occasionally shifted its matches to the United Arab Emirates (UAE) to ensure the safety of players and staff. The UAE has served as a reliable backup venue, hosting IPL matches in 2020 and 2021, showcasing the league's adaptability and resilience.


In [28]:
from langchain.retrievers import ContextualCompressionRetriever  
from langchain.retrievers.document_compressors import LLMChainExtractor  

#### to get only relavent answers in less words we use contextual_trrival 

In [34]:

# Initialize the Chroma DB connection
db_connections = Chroma(embedding_function=model_emb, persist_directory="./chroma_db1")


query = "In which years were IPL matches hosted in the United Arab Emirates?"

# compressor
compressor = LLMChainExtractor.from_llm(chat)

#  contextual compression retriever
compression_retrieval = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=db_connections.as_retriever())

# Get relevant documents using the contextual compression retriever
similar_docs = compression_retrieval.get_relevant_documents(query)

print(similar_docs[0].page_content)

hosting IPL matches in 2020 and 2021


In [56]:
from langchain_community.vectorstores import Chroma 
from langchain.retrievers import ContextualCompressionRetriever
from langchain.chains import ConversationalRetrievalChain
from langchain_openai import ChatOpenAI
from langchain import OpenAI
from langchain.embeddings import OpenAIEmbeddings

# Load the documents into a Chroma vector store
documents = [
    "The Lord of the Rings is a book series written by J.R.R. Tolkien. The series consists of three books: The Fellowship of the Ring, The Two Towers, and The Return of the King.",
    "The Hobbit is a fantasy novel written by J.R.R. Tolkien. It tells the story of Bilbo Baggins and his adventures with dwarves and wizards.",
    "Harry Potter is a book series written by J.K. Rowling. It follows the story of a young wizard named Harry Potter and his friends at the Hogwarts School of Witchcraft and Wizardry.",
    "The Lord of the Rings movies were directed by Peter Jackson and released between 2001 and 2003. The movies were highly successful and won multiple Academy Awards."
]

# Initialize the embedding function
embedding_function = OpenAIEmbeddings()

# Create the Chroma vector store with the embedding function
vectorstore = Chroma.from_texts(documents, embedding=embedding_function)

# Initialize the chat model
chat=ChatOpenAI(openai_api_key=secrete_key) 
# chat = ChatOpenAI(model_name="gpt-4")

# Create a contextual compression retriever and a conversational retrieval chain
compressor = ContextualCompressionRetriever(base_compressor=chat, base_retriever=vectorstore.as_retriever())
qa = ConversationalRetrievalChain.from_llm(llm=chat, retriever=compressor)

# Ask a series of questions
chat_history = []
query = "What is the Lord of the Rings series about?"
result = qa({"query": query, "chat_history": chat_history})
print(result['result'])
chat_history.append((query, result['result']))

query = "How many books are in the series?"
result = qa({"query": query, "chat_history": chat_history})
print(result['result'])
chat_history.append((query, result['result']))
